# Zadania – 3_1 LangChain Chains

*Prepared: 2025-10-04 10:00*

Budowa różnych typów chains: prosty, sekwencyjny, równoległy i mini‑RAG.

> **Instrukcja:** Zadania są krótkie i polegają na uzupełnieniu brakujących fragmentów kodu (`____`). Niektóre komórki zawierają komentarze `# TODO` i `# HINT`. Uruchamiaj komórki po kolei.

## 📦 Wymagania
- `langchain_core`, `langchain_openai`, `langchain` ≥ 0.2
- Opcjonalnie: `faiss-cpu`, `tiktoken`, `python-dotenv`

Jeśli nie masz klucza API, możesz uruchamiać komórki „na sucho” – celem jest *uzupełnienie kodu*, a nie koniecznie jego wykonanie.

### Zadanie 1: Prosty chain (Prompt ➜ LLM)
Uzupełnij brakujące elementy, aby zbudować najprostszy łańcuch, który odpowiada grzecznie po polsku.

In [ ]:

# TODO: Uzupełnij importy i miejsca oznaczone ____
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.output_parsers import StrOutputParser  # opcjonalnie

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)  # lub inny model
prompt = ChatPromptTemplate.from_messages([
    ("system", "Jesteś pomocnym asystentem. Odpowiadaj po polsku."),
    ("user", "{pytanie}")
])

chain = prompt | llm   # | StrOutputParser()  # opcjonalnie

# Oczekiwany efekt: odpowiedź po polsku
wynik = chain.invoke({"pytanie": "W jednym zdaniu: czym jest LangChain?"})
print(type(wynik), "\n", wynik)


### Zadanie 2: Sekwencyjny chain
Stwórz łańcuch dwuetapowy: (1) parafraza, (2) skrócenie do 1 zdania.

In [ ]:

# TODO: Zbuduj chain sekwencyjny prompt1 -> llm -> prompt2 -> llm
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

prompt1 = ChatPromptTemplate.from_messages([
    ("system", "Przekształć wypowiedź użytkownika w bardziej formalny styl. Po polsku."),
    ("user", "{tekst}")
])

prompt2 = ChatPromptTemplate.from_messages([
    ("system", "Streść poprzedni tekst do jednego zdania. Po polsku."),
    ("user", "{wejscie}")
])

# HINT: Użyj operatora | i mapowania kluczy za pomocą .map() lub prostego lambda
stage1 = prompt1 | llm
# poniżej wstaw przekazanie wyniku stage1 do prompt2 (klucz 'wejscie'):
stage2_input_map = lambda x: {"wejscie": x.content if hasattr(x, "content") else str(x)}

chain = stage1 | stage2_input_map | (prompt2 | llm)

print(chain.invoke({"tekst": "LangChain pozwala budować złożone przepływy z LLM."}))


### Zadanie 3: Mini‑RAG chain (retriever ➜ kontekst ➜ odpowiedź)
Uzupełnij brakujące fragmenty, aby zbudować prosty łańcuch RAG z lokalnym indeksem FAISS.

In [ ]:

# TODO: Zbuduj mini‑RAG. Jeśli nie masz faiss, możesz tylko uzupełnić kod bez uruchamiania.
# HINT: Teksty -> embeddingi -> wektorowy index -> retriever -> prompt -> llm
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

try:
    from langchain_community.vectorstores import FAISS
    ok = True
except Exception as e:
    print("Brak FAISS, ale możesz uzupełnić kod. Info:", e)
    ok = False

docs = [
    "LangChain pozwala łączyć modele z narzędziami i pamięcią.",
    "RAG to wzorzec: retrieval augmented generation.",
    "FAISS to biblioteka do wyszukiwania podobnych wektorów."
]

emb = OpenAIEmbeddings()  # wymaga API; traktuj jako placeholder
if ok:
    db = FAISS.from_texts(docs, embedding=emb)
    retriever = db.as_retriever(k=2)
else:
    retriever = lambda q: []  # placeholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "Użyj podanego kontekstu do odpowiedzi. Odpowiadaj po polsku."),
    ("system", "Kontekst: {context}"),
    ("user", "{question}")
])

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

def build_context(question: str):
    # HINT: wywołaj retriever i połącz dokumenty w jeden string
    ctx_docs = retriever(question) if callable(retriever) else []
    if hasattr(ctx_docs, "__iter__"):
        texts = [getattr(d, "page_content", str(d)) for d in ctx_docs]
    else:
        texts = []
    return "\n---\n".join(texts) if texts else "Brak kontekstu (placeholder)."

rag = (
    {"context": lambda x: build_context(x["question"]), "question": lambda x: x["question"]}
    | prompt
    | llm
)

print(rag.invoke({"question": "Czym jest RAG i do czego służy FAISS?"}))
